In [1]:
import time
import requests
import pandas as pd
from tqdm import tqdm

In [130]:
headers_text = """Accept:text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8
Accept-Encoding:gzip, deflate, sdch
Accept-Language:zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4,zh-CN;q=0.2,ko;q=0.2
Cache-Control:max-age=0
Connection:keep-alive
Cookie:_free_proxy_session=BAh7B0kiD3Nlc3Npb25faWQGOgZFVEkiJWVhMTM4YjMxN2E4MGZkZGJmYmUyNDhlZDIyODc0M2FmBjsAVEkiEF9jc3JmX3Rva2VuBjsARkkiMTRYaTIvMllyTFV5NTByeWhLTGF3U0R4MHZTQmN1RHZKa3dBTzR3UlFNN289BjsARg%3D%3D--b0a39c732f63bdd1e8ea0473bc271dbc06b1b2df
DNT:1
Host:www.xicidaili.com
If-None-Match:W/"773c738d73035aef19b0b308957ccf02"
Referer:http://www.xicidaili.com/wn/
Upgrade-Insecure-Requests:1
User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"""

def parse_headers(t):
    lines = [line for line in t.strip().split('\n')]
    return {line.split(':')[0]: line.split(':')[1] for line in lines}


headers = parse_headers(headers_text)

In [132]:
# get proxies

def get_proxy_df(page):
    url = 'http://www.xicidaili.com/nn/{}'.format(page)
    res = requests.get(url, headers=headers)
    df = pd.read_html(res.text)[0].iloc[1:, [1, 2, 4, 5, 8, 9]]
    df.columns = ['ip', 'port', 'anonymous_level', 'http_type', 'live_time', 'valid_date']
    df['proxy'] = df['ip'] + ':' + df['port']
    del df['ip'], df['port']
    time.sleep(2)
    return df

frame = []
for page in tqdm(range(1, 11)):
    frame.append(get_proxy_df(page))

proxy_df = pd.concat(frame).reset_index(drop=True)
proxy_df


  0%|          | 0/10 [00:00<?, ?it/s]


ConnectionError: HTTPConnectionPool(host='www.xicidaili.com', port=80): Max retries exceeded with url: /nn/1 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x119cf0cf8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [74]:
def validate_proxy(proxy):
    try:
        res = requests.get('http://icanhazip.com', 
                           headers=headers, 
                           proxies={'http': proxy}, timeout=5)
    except:
        return 'connect_failed'
    
    try:    
        assert len(res.text.split('\n')) == 2
        assert len([e for e in res.text if e=='.']) == 3
    except:
        print(res.text)
        return 'not_valid_ip'
    
    return res.text

In [93]:
proxies = proxy_df['proxy']

In [26]:
import grequests

In [30]:
proxies[:5]

0    114.235.81.48:8118
1     59.62.165.127:808
2     60.167.133.76:808
3    182.88.41.158:8123
4     116.28.120.25:808
Name: proxy, dtype: object

In [124]:
%%time

rs = (grequests.get('http://icanhazip.com', proxies={'http': proxy}, timeout=5) for proxy in proxies[:100])

valid_ips = []
for r in grequests.map(rs, size=20):
    try:
        valid_ips.append(r.text.strip())
    except (AttributeError, AssertionError) as e:
        pass


CPU times: user 4.78 s, sys: 797 ms, total: 5.57 s
Wall time: 11min 39s


In [125]:
# Too many redirects
# https://stackoverflow.com/questions/23651947/python-requests-requests-exceptions-toomanyredirects-exceeded-30-redirects

def get_518(proxy):
    s = requests.Session()
    s.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
    res = s.get('https://case.518.com.tw', proxies={'http': proxy}, timeout=5)
    return res.text

In [126]:
%%time

for p in proxies:
    if p.split(':')[0] in valid_ips:
        try:
            print(len(get_518(p)))
        except:
            print('failed!')
        
        time.sleep(1)

105890
105902
105892
105911
105911
105890
105892
105902
105911
105907
105907
105902
105906
105902
105911
105890
105902
105906
105892
105902
105907
105902
105902
105892
105911
105906
105906
105892
105911
105892
105902
105907
105911
CPU times: user 979 ms, sys: 87.3 ms, total: 1.07 s
Wall time: 42 s


In [129]:
len([p for p in proxies if p.split(':')[0] in valid_ips])

33